# Pytorch Correlsted Gaussian Wave function 

Optimize a wave function expanded in correlated gaussian basis functions.


## Integral formulas  (Matrix Elements)

### Skl (Overlap), Tkl (Kinetic Energy), Vkl (Potential Energy) Hkl (Hamiltonion)

 **py_matel**: returnes symmetry projected matrix elements in a
 basis of simple correlated gaussians phi_kl = exp[-r'(Lk*Ll' kron I3)r]

 * n:      the number of "psuedo" particles i.e. Center of mass translationl degrees of freedom are removed so n is N-1 if N is the number of "real" particles.
            
 * vechLk: nonlinear exponent parameters n(n+1)/2 x 1
 * vechLl:    These will form the lower triangle matrices Lk and Ll


 * Sym:     symmetry projection matrix for the term being computed
 * Mass:    mass matrix for kinetic energy (reduced masses of particles )
 * vecQ:    charge products for potential energy (elements q_i x q_j where q are particle charges)

In [67]:
import numpy as np
from numpy.linalg import inv, det
from numpy import trace, abs, sqrt, zeros, pi

import torch

In [108]:
def py_matel(n, vechLk, vechLl, Sym, Mass, vecQ):
    
    # initialize arrays
    Lk=zeros((n,n));
    Ll=zeros((n,n));
    Ak=zeros((n,n));
    Al=zeros((n,n));
    Akl=zeros((n,n));
    invAkl=zeros((n,n));
    invAk=zeros((n,n));
    invAl=zeros((n,n));
    
    # build Lk and Ll

    #count=0;
    #for j=1:n
    #  for i=j:n
    #    count=count+1;
    #    Lk(i,j) = vechLk(count);
    #    Ll(i,j) = vechLl(count);
    #  end
    #end
    
    idxu = np.triu_indices(n);
    Lk[idxu] = vechLk;
    Ll[idxu] = vechLl;
    Lk=Lk.T
    Ll=Ll.T
    
    # apply symmetry projection on Ll
    
    Ll = Sym.T @ Ll;
    
    # build Ak, Al, Akl, invAkl, invAk, invAl

    Ak = Lk@Lk.T;
    Al = Ll@Ll.T;
    Akl = Ak+Al;
    
    invAkl = inv(Akl);
    invAk  = inv(Ak);
    invAl  = inv(Al);
    
    # Overlap: (normalized)
    skl = 2**(3*n/2) * sqrt( (abs(det(Lk))*abs(det(Ll))/det(Akl) )**3 );

    # kinetic energy

    tkl = skl*(6*trace(Mass@Ak@invAkl@Al));
    
    # potential energy
    
    RIJ = zeros((n,n));
    # 1/rij i~=j
    for j in range(0,n-1):
        for i in range(j+1,n):
            tmp2 = invAkl[i,i] + invAkl[j,j] - 2*invAkl[i,j];
            RIJ[i,j] = 2/sqrt(pi) * skl/sqrt(tmp2);


    # 1/rij i=j
    for i in range(0,n):
        RIJ[i,i] = 2/sqrt(pi) * skl/sqrt(invAkl[i,i]);
    
    Q =zeros((n,n))
    Q[idxu] = vecQ;
    Q=Q.T
    vkl = np.sum(RIJ*Q)

    hkl = tkl + vkl
    
    return {'skl':skl, 'tkl':tkl, 'vkl':vkl, 'hkl':hkl}


In [109]:
def test_matel():
    n = 3;
    vechLk = np.array([  1.00000039208682, 
              0.02548044275764261, 
              0.3525161612610669,
              1.6669144815242515,
              0.9630555318946559,
              1.8382882034659822 ]);
    
    vechLl = np.array([  1.3353550436464964,
               0.9153272033682132,
               0.7958636766525028,
               1.8326931436447955,
               0.3450426931160630,
               1.8711839323167831 ]);
    
    Sym = np.array([[0,0,1],
                    [0,1,0],
                    [1,0,0]]);
    
    Mass = np.array([[5.446170e-4, 2.723085077e-4, 2.723085077e-4],
                     [2.723085077e-4, .5002723085, 2.723085077e-4],
                     [2.723085077e-4, 2.723085077e-4, .5002723085 ]]);
    
    vecQ = [1, -1, -1, -1, 1, -1];
    
    matels = py_matel(n, vechLk, vechLl, Sym, Mass, vecQ)
    
    print('skl: ',matels['skl'])
    print('tkl: ',matels['tkl'])
    print('vkl: ',matels['vkl'])
    print('hkl: ',matels['hkl'])
    

In [110]:
test_matel()

skl:  0.5333557299037264
tkl:  4.3509068790883045
vkl:  -2.3839605567755693
hkl:  1.9669463223127353


## Energy Calculation

 **energyrc**: returns the energy Rayleigh quotient c'Hc/c'Sc
 in a basis of simple correlated Gaussians. The minimum of hte Rayleigh quotent 
 is the minimum of the smallest eigenvalue of the matrix representation of the Schrodinger euqation,
 (H-eS)c = 0 It is simpiler to compute than the full set of eigenvalues and verctors. A good optimization will determine the linear coeffients c i.e the eigenvector of the enery e.

* x:		the optimization parameters
* n:		the number of pseudo particles (size of Lk)
* nb:		the number of basis functions

* the first n(n+1)/2 * nb elements of x are exponent parameters (vechLk)
* the last nb elements of x are the linear coeff's c

In [124]:
def py_energyrc(x,n,nb,Mass,Charge,Sym,symc):
    
    nx = len(x);
    nn = int(n*(n+1)/2);
    nsym = len(symc);
    
    # Build H and S
    H = zeros((nb,nb));
    S = zeros((nb,nb));
    T = zeros((nb,nb));
    V = zeros((nb,nb));
    
    # outer loop is over symmetry terms
    for k in range(0,nsym):
        
        for j in range(0,nb):
            for i in range(j,nb):
                idxi = i*nn;
                idxj = j*nn;
                
                vechLi = x[idxi:idxi+nn];
                vechLj = x[idxj:idxj+nn];
                
                matels = py_matel(n,vechLi,vechLj,Sym[:,:,k],Mass,Charge);
                
                S[i,j] = S[i,j] + symc[k]*matels['skl'];
                T[i,j] = T[i,j] + symc[k]*matels['tkl'];
                V[i,j] = V[i,j] + symc[k]*matels['vkl'];
    H = T + V
    
    # complete uppertriangle of H and S
    for i in range(0,nb):
        for j in range(i+1,nb):
            H[i,j] = H[j,i];
            S[i,j] = S[j,i];
            
    # and the energy is:
    c = x[-nb:];
    cHc = c.T@H@c;
    cSc = c.T@S@c;
    eng = cHc/cSc;
    return eng
                

In [127]:
def test_energyrc():
    Mass = np.array([[0.5, 0.0, 0.0],
                     [0.0, 0.5, 0.0],
                     [0.0, 0.0, 0.5]]);
    
    Charge = np.array([-3, 1, 1, -3, 1, -3]);
    
    # symmetry projection terms
    Sym = zeros((3,3,6))
    # (1)(2)(3)
    Sym[:,:,0] = np.array([[1,0,0],[0,1,0],[0,0,1]]);
    # (12)
    Sym[:,:,1] = np.array([[0,1,0],[1,0,0],[0,0,1]]);
    # (13)
    Sym[:,:,2] = np.array([[0,0,1],[0,1,0],[1,0,0]]);
    # (23)
    Sym[:,:,3] = np.array([[1,0,0],[0,0,1],[0,1,0]]);
    # (123)
    Sym[:,:,4] = np.array([[0,1,0],[0,0,1],[1,0,0]]);
    # (132)
    Sym[:,:,5] = np.array([[0,0,1],[1,0,0],[0,1,0]]);

    # coeff's
    symc = np.array([4.0,4.0,-2.0,-2.0,-2.0,-2.0]);

    n=3;
    nb=8;
    
    xvechL=np.array([
     1.6210e+00,
    -2.1504e-01,
     9.0755e-01,
     9.7866e-01,
    -2.8418e-01,
    -3.5286e+00,
    -3.3045e+00,
    -4.5036e+00,
    -3.2116e-01,
    -7.1901e-02,
     1.5167e+00,
    -8.4489e-01,
    -2.1377e-01,
    -3.6127e-03,
    -5.3774e-03,
    -2.1263e+00,
    -2.5191e-01,
     2.1235e+00,
    -2.1396e-01,
    -1.4084e-03,
    -1.0092e-02,
     4.5349e+00,
     9.4837e-03,
     1.1225e+00,
    -2.1315e-01,
     5.8451e-02,
    -4.9410e-03,
     5.0853e+00,
     7.3332e-01,
     5.0672e+00,
    -2.1589e-01,
    -6.8986e-03,
    -1.4310e-02,
     1.5979e+00,
     3.3946e-02,
    -8.7965e-01,
    -1.1121e+00,
    -2.1903e-03,
    -4.6925e-02,
     2.1457e-01,
     3.3045e-03,
     4.5120e+00,
    -2.1423e-01,
    -1.6493e-02,
    -2.3429e-03,
    -8.6715e-01,
    -6.7070e-02,
     1.5998e+00
     ])
    
    evec = np.array([
      -6.0460e-02,
       7.7708e-05,
       1.6152e+00,
       9.5443e-01,
       1.1771e-01,
       3.2196e+00,
       9.6344e-01,
       3.1398e+00
    ])
    
    x1 = np.concatenate((xvechL,evec))
    
    #print(x1)
    energy = py_energyrc(x1,n,nb,Mass,Charge,Sym,symc)
    print(energy)

In [128]:
test_energyrc()

-7.36153159192785


Exception in callback BaseAsyncIOLoop._handle_events(15, 1)
handle: <Handle BaseAsyncIOLoop._handle_events(15, 1)>
Traceback (most recent call last):
  File "/home/kinghorn/anaconda3/envs/pytorch/lib/python3.6/asyncio/events.py", line 145, in _run
    self._callback(*self._args)
  File "/home/kinghorn/anaconda3/envs/pytorch/lib/python3.6/site-packages/tornado/platform/asyncio.py", line 117, in _handle_events
    handler_func(fileobj, events)
  File "/home/kinghorn/anaconda3/envs/pytorch/lib/python3.6/site-packages/tornado/stack_context.py", line 276, in null_wrapper
    return fn(*args, **kwargs)
  File "/home/kinghorn/anaconda3/envs/pytorch/lib/python3.6/site-packages/zmq/eventloop/zmqstream.py", line 450, in _handle_events
    self._handle_recv()
  File "/home/kinghorn/anaconda3/envs/pytorch/lib/python3.6/site-packages/zmq/eventloop/zmqstream.py", line 480, in _handle_recv
    self._run_callback(callback, msg)
  File "/home/kinghorn/anaconda3/envs/pytorch/lib/python3.6/site-packages/